In [ ]:
import sys
import os
import json

sys.path.insert(0, 'compiled_protobufs')

from draw import get_taksgraph_visualization
from google.protobuf.json_format import MessageToDict
import json
from google.protobuf.json_format import Parse
from pyserini.search import LuceneSearcher

from taskmap_pb2 import TaskMap

## Annotation Notebook

In [ ]:
import time

def get_taskgraph(id, domain):
    # print(os.path.join(os.getcwd(), "indexes", domain, "system_index_sparse"))
    taskmap_index_path = os.path.join(os.getcwd(), "indexes", domain, "system_index_sparse")
    searcher = LuceneSearcher(index_dir=taskmap_index_path)
    id = id.replace('\n','')
    doc = searcher.doc(docid=id)
    taskmap_json = json.loads(doc.raw())['recipe_document_json']
    taskmap = Parse(json.dumps(taskmap_json), TaskMap())
    return taskmap
    

# domains = []
domains = ["cooking,", "diy"]
k = 10
for domain in domains:
    empty_annotations = f"measurements/{domain}/empty_annotations/empty_annotations-{k}.csv"
    qrels = f"measurements/judgments/{domain}.qrels"
    with open(qrels, "r") as f_qrles:
        qrles = [q for q in f_qrles]
        done_annoations_count = len(qrles)
        print(f"Number of annotations for {domain}:", len(qrles))
    with open(empty_annotations, "r") as f_empty:
        annotations = [annotation for annotation in f_empty]
        annotaitons_count = len(annotations)

    annotations_tracker = done_annoations_count
    for annotation in annotations[done_annoations_count+1:]:

        annnotation = annotations[annotations_tracker+1]
        query_id, target_query, doc_id = annnotation.split(",")
        doc_id = doc_id.replace('\n','')
        taskmap = get_taskgraph(doc_id, domain)
        
        print(f"Annotation for {domain} number {annotations_tracker}/{annotaitons_count}", flush = True)
        print("Query: ", target_query , flush = True)
        print(taskmap.title, taskmap.source_url, flush = True)
        time.sleep(0.05)

        rating = int(input("Relevance rating:"))
        print("Rating:", rating)
        with open(qrels, "a") as f_qrles:
            # print(query_id, target_query, id)
            f_qrles.write(f"{query_id} Q0 {doc_id} {rating}\n")
        annotations_tracker += 1
        

        


In [ ]:
# convert csv to qrels

path_from = "/home/ubuntu/task-search-quality/measurements/cooking/filled_annotations/cooking_judgments.csv"
path_to = "/home/ubuntu/task-search-quality/measurements/judgments/cooking-all.qrels"


In [ ]:
with open(path_from, "r") as f:
    rows = [r.replace('\n','').split(',') for r in f]

qrels = []
for row in rows:
    qrels.append(f"{row[0]} Q0 {row[1]} {rows[3]}\n")

with open(path_to, "w") as f:
    f.writelines(qrels)
